In [1]:
import numpy as np
import pandas as pd

/tmp/ipykernel_24722/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
gamma = 0.9
alpha = 0.6
door = -20
wall = -200
home = 1000

In [3]:
rewardsTable = np.array([
    [None,-1,None,None,wall,None,None,None,None,None,None,None,None,None,None,None],
    [-1,None,-1,None,None,door,None,None,None,None,None,None,None,None,None,None],
    [None,-1,None,-1,None,None,wall,None,None,None,None,None,None,None,None,None],
    [None,None,-1,None,None,None,None,-1,None,None,None,None,None,None,None,None],
    [-1,None,None,None,None,door,None,None,-1,None,None,None,None,None,None,None],
    [None,-1,None,None,wall,None,wall,None,None,-1,None,None,None,None,None,None],
    [None,None,-1,None,None,door,None,-1,None,None,-1,None,None,None,None,None,None],
    [None,None,None,-1,None,None,wall,None,None,None,None,-1,None,None,None,None],
    [None,None,None,None,wall,None,None,None,None,-1,None,None,-1,None,None,None],
    [None,None,None,None,None,door,None,None,-1,None,-1,None,None,home,None,None],
    [None,None,None,None,None,None,wall,None,None,-1,None,-1,None,None,-1,None],
    [None,None,None,None,None,None,None,-1,None,None,-1,None,None,None,None,-1],
    [None,None,None,None,None,None,None,None,-1,None,None,None,None,home,None,None],
    [None,None,None,None,None,None,None,None,None,-1,None,None,-1,None,-1,None],
    [None,None,None,None,None,None,None,None,None,None,-1,None,None,home,None,-1],
    [None,None,None,None,None,None,None,None,None,None,None,-1,None,None,-1,None]
    ], dtype='object')

In [4]:
coordinatesToStates = {
    'A1' : 0,
    'B1' : 1,
    'C1' : 2,
    'D1' : 3,
    'A2' : 4,
    'B2' : 5,
    'C2' : 6,
    'D2' : 7,
    'A3' : 8,
    'B3' : 9,
    'C3' : 10,
    'D3' : 11,
    'A4' : 12,
    'B4' : 13,
    'C4' : 14,
    'D4' : 15,
}


In [5]:
possibleActions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]


In [17]:
stateToCoordinates = {state: coordinates for coordinates, state in coordinatesToStates.items()}

def policy(stateT, legalActions, QValues, method, epsilon):
    
    if method == 'QLearning':
        return np.random.choice(legalActions)
    if method == 'SARSA':
        if np.random.random() < epsilon:
            return np.random.choice(legalActions)
        else:
            #print('Legal actions: ', legalActions)
            bestAction = legalActions[0]
            for index in legalActions:
                #print(index, QValues[stateT][index])
                if QValues[stateT][index] > QValues[stateT][bestAction]:
                    bestAction = index
            #print('Best Action: ', bestAction)
            #print("QValues: ", QValues[stateT].astype(float))
            return bestAction

def legalActionChecker(index, rewards, state):
    legalActions = []
    i = 0
    while i <= index:
        if rewards[state][i] != None:
            legalActions.append(i)
        i+=1
    return legalActions

def path(QValues, startCoord = 'A1', endCoord = 'B4'):
    path = [startCoord]
    reward = 0
    nextCoord = startCoord
    while (nextCoord != endCoord):
        startState = coordinatesToStates[startCoord]
        nextState = np.argmax(QValues[startState,])
        nextCoord = stateToCoordinates[nextState]
        reward += rewardsTable[startState][nextState]
        path.append(nextCoord)
        startCoord = nextCoord
    return {'Reward': reward, 'Path':path}

def qLearning(episodeCount, method, epsilon):
    currentEpisode = 0
    QValues = np.array(np.zeros([16,16]))

    while currentEpisode < episodeCount:
        stateT = 0
        stateNext = int
        legalActions = []
        reward = 0
        i = 0
        steps = 0
        #While haven't reached the terminal state
        while stateNext != 13 and steps <= 50:
            legalActions = legalActionChecker(15,rewardsTable,stateT)
            print('Step',steps, 'is ', stateT)
            #print(QValues)
            pd.DataFrame(QValues.astype(int), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
            stateNext= policy(stateT, legalActions,QValues,method,epsilon)
            reward += rewardsTable[stateT][stateNext]
            TD = rewardsTable[stateT][stateNext] + gamma * QValues[stateNext][np.argmax(QValues[stateNext])] - QValues[stateT][stateNext]
            QValues[stateT][stateNext] += alpha*TD 
            stateT=stateNext
            legalActions = []
            i = 0
            steps+= 1
        currentEpisode += 1
    return QValues


In [18]:
stateToCoordinates = {state: coordinates for coordinates, state in coordinatesToStates.items()}

In [19]:
#pd.DataFrame(qLearning(1,'QLearning', 0.5).astype(int), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])


#print(qLearning(15,'QLearning', 0.5).astype(int))
#print(qLearning(15,'SARSA', 0.5).astype(int))
#print(path())

In [20]:
pd.DataFrame(qLearning(1,'SARSA', 0.3), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])


Step 0 is  0
Step 1 is  1
Step 2 is  0
Step 3 is  4
Step 4 is  0
Step 5 is  1
Step 6 is  2
Step 7 is  1
Step 8 is  5
Step 9 is  1
Step 10 is  0
Step 11 is  1
Step 12 is  2
Step 13 is  3
Step 14 is  7
Step 15 is  3
Step 16 is  7
Step 17 is  11
Step 18 is  7
Step 19 is  6
Step 20 is  2
Step 21 is  6
Step 22 is  5
Step 23 is  9
Step 24 is  5
Step 25 is  4
Step 26 is  5
Step 27 is  6
Step 28 is  7
Step 29 is  3
Step 30 is  2
Step 31 is  1
Step 32 is  0
Step 33 is  1
Step 34 is  5
Step 35 is  1
Step 36 is  2
Step 37 is  3
Step 38 is  2
Step 39 is  1
Step 40 is  0
Step 41 is  1
Step 42 is  2
Step 43 is  6
Step 44 is  10
Step 45 is  6
Step 46 is  2
Step 47 is  3
Step 48 is  7
Step 49 is  11
Step 50 is  10


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0000,-0.98976,0.0000,0.000,-120.0,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,-0.9744,0.00000,-0.9744,0.000,0.0,-16.8,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,0.0000,-0.93600,0.0000,-0.936,0.0,0.0,-168.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,0.0000,0.00000,-0.8400,0.000,0.0,0.0,0.0,-0.936,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,-0.6000,0.00000,0.0000,0.000,0.0,-12.0,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,0.0000,-0.84000,0.0000,0.000,-120.0,0.0,-120.0,0.000,0.0,-0.6,0.0,0.00,0.0,0.0,0.0,0.0
6,0.0000,0.00000,-0.8400,0.000,0.0,-12.0,0.0,-0.600,0.0,0.0,-0.6,0.00,0.0,0.0,0.0,0.0
7,0.0000,0.00000,0.0000,-0.840,0.0,0.0,-120.0,0.000,0.0,0.0,0.0,-0.84,0.0,0.0,0.0,0.0
8,0.0000,0.00000,0.0000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,0.0000,0.00000,0.0000,0.000,0.0,-12.0,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [10]:
#print(qLearning(20000,'QLearning', 0.5).astype(int))